In [56]:
import sys, os, time
import pandas as pd
import numpy as np
import vcf
import random
from collections import OrderedDict, Counter
from operator import itemgetter
from collections import defaultdict
from __future__ import division
import math
import os.path as op
from os import listdir as ls
from ipyparallel import Client

In [410]:
?os.path.dirname

In [3]:
home_dir  = '/home/lindb/wbp/bayenv2/nullruns/'

In [4]:
imputedData = pd.read_csv('/home/lindb/wbp/bayenv2/imputed_z12_maf_swp_trans_z12.txt',header=0,index_col=0,sep="\t")
imputedData.head()

,NODE_1000013_length_91_cov_1.802198_37,NODE_1000031_length_98_cov_2.000000_30,NODE_1000037_length_93_cov_2.043011_33,NODE_1000042_length_93_cov_1.118280_100,NODE_1000053_length_94_cov_1.744681_25,NODE_1000061_length_47_cov_1.680851_49,NODE_1000118_length_95_cov_2.000000_24,NODE_1000128_length_98_cov_1.714286_53,NODE_1000180_length_98_cov_2.204082_17,NODE_1000188_length_67_cov_1.223881_65,...,NODE_999880_length_98_cov_1.683674_22,NODE_999886_length_96_cov_1.020833_38,NODE_999899_length_96_cov_1.000000_91,NODE_999927_length_95_cov_2.000000_55,NODE_999933_length_76_cov_1.236842_76,NODE_999938_length_98_cov_1.142857_98,NODE_999941_length_107_cov_1.196262_25,NODE_999958_length_91_cov_1.021978_17,NODE_999968_length_75_cov_1.680000_37,NODE_999969_length_92_cov_4.271739_37
009compiled,0,1,0,0,0,0,1,0,1,0,...,0,0,1,0,1,1,0,0,0,1
010compiled,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
011compiled,1,0,1,0,0,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0
012compiled,0,0,0,1,0,0,1,0,1,0,...,0,1,2,0,1,0,0,0,1,1
013compiled,2,0,0,0,0,0,1,1,1,0,...,0,0,1,0,1,0,0,0,0,1


In [5]:
stpDF = pd.read_csv('/home/lindb/wbp/sampsTOpop.txt',header=0,sep="\t")
stpDF.head()

,sampID,pop,plot
0,028compiled,Dicks_Pass,3
1,024compiled,Dicks_Pass,2
2,020compiled,Dicks_Pass,2
3,025compiled,Dicks_Pass,2
4,023compiled,Dicks_Pass,2


In [6]:
len(imputedData.index),len(stpDF.index)

(244, 244)

In [8]:
pops = np.unique(stpDF['pop']).tolist()
pops

['Dicks_Pass',
 'Freel_Peak',
 'Heavenly',
 'Little_Round_Top',
 'Mt_Rose_Ophir',
 'Rifle_Peak',
 'Snow_Valley_Peak',
 'West_Shore_Peaks']

In [11]:
popDict = Counter()
for row in stpDF.index:
    POP = stpDF.loc[row,'pop']
    popDict[POP] += 1
for pop in popDict.keys():
    print pop,popDict[pop]

Dicks_Pass 25
Freel_Peak 48
Heavenly 25
Little_Round_Top 25
Mt_Rose_Ophir 49
Rifle_Peak 24
Snow_Valley_Peak 24
West_Shore_Peaks 24


In [13]:
sums = 0
for pop in popDict.keys():
    sums = sums + popDict[pop]
sums

244

In [77]:
for i in range(100): #make 100 runfiles to make SNPSFILES
    num = str(i).zfill(3)
    text = '''print "starting"
import sys
import os, time
import pandas as pd
import numpy as np
import scipy as sp
import random
from collections import OrderedDict
from operator import itemgetter
import random

filE = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/%supdate.txt'
with open(filE,'w') as o:
    text = 'importing data\\n'
    o.write("%%s" %% text)
imputedData = pd.read_csv('/home/lindb/wbp/bayenv2/imputed_z12_maf_swp_trans_z12.txt',header=0,index_col=0,sep="\\t")
print "done importing"

stpDF = pd.read_csv('/home/lindb/wbp/sampsTOpop.txt',header=0,sep="\\t")

stpnew = stpDF

pops = np.unique(stpnew['pop']).tolist()

popDict = OrderedDict()
for pop in pops:
    popDict[pop] = 0

for row in stpnew.index:
    POP = stpDF.loc[row,'pop']
    popDict[POP] += 1

pops = []
for pop in sorted(popDict.keys()):
    pops.append(pop)

print "creating idx"
with open(filE,'a') as o:
    text = 'creating idx\\n'
    o.write("%%s" %% text)
idx = range(len(imputedData.index))
sampDict = OrderedDict()
print "assigning random samps to pops"
with open(filE,'a') as o:
    text = 'assigning random samps to pops\\n'
    o.write("%%s" %% text)
for pop in sorted(popDict.keys()): #randomly assign samples to pops
    which = random.sample(idx,popDict[pop])
    samps = itemgetter(*which)(imputedData.index.tolist())
    sampDict[pop] = imputedData[imputedData.index.isin(samps)]
    idx = [x for x in idx if x not in which]

print "iterating loci"
with open(filE,'a') as o:
    text = 'iterating loci\\n'
    o.write("%%s" %% text)
locCount = 0
nullDict = OrderedDict()
for locus in imputedData.columns:
    nullDict[locus] = OrderedDict()
    
    for pop in sorted(pops):
        
        zero = sampDict[pop][locus].tolist().count(0) #count the first homozygotes
        one  = sampDict[pop][locus].tolist().count(1) #count the heterozygotes
        two  = sampDict[pop][locus].tolist().count(2) #count the second homozygotes        
        A1 = 2*zero + one
        A2 = 2*two + one
        if len(nullDict[locus].keys()) == 0:
            nullDict[locus]['A1'] = OrderedDict()
            nullDict[locus]['A2'] = OrderedDict()
        nullDict[locus]['A1'][pop] = A1
        nullDict[locus]['A2'][pop] = A2
    locCount += 1
    if locCount %% 1000 == 0:
        print locCount
        with open(filE,'a') as o:
            text = "%%s\\n" %% str(locCount)
            o.write("%%s" %% text)
        #break
    #break
        
print "writing file, iteration %s"
filE = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/UnbinnedImputedSNPSFILE%s_HEADERROW.txt' 
DIR = os.path.dirname(filE)
if not os.path.exists(DIR):
    os.makedirs(DIR)
with open(filE, 'w') as o:
    line = '\\t'.join([str(pop) for pop in sorted(popDict.keys())]) + str('\\n')
    o.write("%%s" %% line)
    for locus in nullDict.keys():
        for allele in nullDict[locus].keys():
            line = str(locus) + '\\t' + '\\t'.join([str(x) for x in nullDict[locus][allele].values()]) + str('\\n')
            #print locus,allele,line
            o.write("%%s" %% line)
o.close()
''' % (str(num),str(i).zfill(3),str(i).zfill(3))
    FILE = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEsrunfiles/imp_run%s.py' % str(i).zfill(3)
    if not op.exists(op.dirname(FILE)):
        os.makedirs(op.dirname(FILE))
    with open(FILE,'w') as o:
        o.write(text)
    o.close

In [78]:
root = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEsrunfiles/'
files = ls(root)
for f in files:
    runname = op.join(root,f).split(".")[0].split("/")[-1][-3:]
    text = '''#!/bin/bash
#$ -N %s
#$ -V
#$ -j y
#$ -cwd

cd /home/lindb/wbp/bayenv2/nullruns/SNPSFILEsrunfiles/
python %s
''' % ('null'+runname,f)
    filE = op.join(root,'null'+runname + '.sh')
    with open(filE,'w') as o:
        o.write('%s' % text)
    o.close()

In [25]:
filE

'/home/lindb/wbp/bayenv2/nullruns/SNPSFILEsrunfiles/null099.sh'

# null matrix estimation

#1 Get list of SNPSFILEs and create a directory using that name
#2 get SNPSFILE with header/idx, save as SNPSFILE without headerIDX in new directory
#3 copy ./bayenv2, ENVIRONFILE to each new folder
#4 create qsub files to create MATRIXFILEs
#5 split the SNPSFILE
#6 make usable matrix files
#7 run bayenv2

In [3]:
snp_dir = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/'

In [28]:
import shutil

In [40]:
op.join(snp_dir,f)

'/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/UnbinnedImputedSNPSFILE_024_HEADERROW.txt'

### Step 1 - Get list of SNPSFILEs and create a directory using that name

In [82]:
#move each of the files into its own directory - eg, 'null000'
os.chdir(snp_dir)
files = ls(snp_dir)

for f in files:
    if f.endswith('txt'):
        splits = f.split("_")
        num = splits[0][-3:]
        DIR = op.join(snp_dir,'null%s' % str(num))
        
        newf = op.join(DIR,f)
        print num,f,'\n%s' % DIR
        
        
        if not op.exists(DIR):
            os.makedirs(DIR)
            
        shutil.move(op.join(snp_dir,f),newf)
        

064 UnbinnedImputedSNPSFILE064_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null064
065 UnbinnedImputedSNPSFILE065_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null065
066 UnbinnedImputedSNPSFILE066_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null066
067 UnbinnedImputedSNPSFILE067_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null067
068 UnbinnedImputedSNPSFILE068_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null068
069 UnbinnedImputedSNPSFILE069_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null069
070 UnbinnedImputedSNPSFILE070_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null070
071 UnbinnedImputedSNPSFILE071_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null071
072 UnbinnedImputedSNPSFILE072_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null072
073 UnbinnedImputedSNPSFILE073_HEADERROW.txt 
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null073
074 UnbinnedImputedSNPSFILE074

In [6]:
dirs = ls(snp_dir)
dirs = sorted([op.join(snp_dir,d) for d in dirs if 'null' in d])
len(dirs)

100

In [85]:
df = pd.read_csv('/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/UnbinnedImputedSNPSFILE000_HEADERROW.txt',
                 header=0,index_col=0,sep="\t")
df.head()

,Dicks_Pass,Freel_Peak,Heavenly,Little_Round_Top,Mt_Rose_Ophir,Rifle_Peak,Snow_Valley_Peak,West_Shore_Peaks
NODE_1000013_length_91_cov_1.802198_37,46,84,45,42,85,43,40,44
NODE_1000013_length_91_cov_1.802198_37,4,12,5,8,13,5,8,4
NODE_1000031_length_98_cov_2.000000_30,44,79,44,37,86,44,38,42
NODE_1000031_length_98_cov_2.000000_30,6,17,6,13,12,4,10,6
NODE_1000037_length_93_cov_2.043011_33,46,86,42,43,85,47,41,38


### Step 2 - get SNPSFILE with header/idx, save as SNPSFILE without headerIDX in new directory

In [106]:
for d in dirs:
    num = d[-3:]
    snpsfile = ls(d)[0]
    df = pd.read_csv(op.join(d,snpsfile),header=0,index_col=0,sep="\t")
    
    filE = op.join(d,'UnbinnedImputedSNPSFILE%s_noHEADERIDX.txt' % num)
    df.to_csv(filE,header=False,index=False,sep="\t")

### Step 3 - copy ./bayenv2, ENVIRONFILE to each new folder

In [110]:
EF = '/home/lindb/wbp/bayenv2/ENVIRONFILE.txt'
bayenv = '/home/lindb/wbp/bayenv2/bayenv2'

In [111]:
for d in sorted(dirs):
    shutil.copy(EF,d)
    shutil.copy(bayenv,d)

In [113]:
import random

In [121]:
random.sample(range(100000),1)

[76588]

### Step 4 - create qsub files to create MATRIXFILEs

In [48]:
df = pd.read_csv('/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/UnbinnedImputedSNPSFILE064_HEADERROW.txt')
df.to_csv('/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/UnbinnedImputedSNPSFILE064_noHEADERIDX.txt',
          header=False,index=False,sep='\t')

In [ ]:
/home/lindb/wbp/bayenv2/bayenv2 -i UnbinnedImputedSNPSFILE064_noHEADERIDX.txt -p 8 -k 100000 -r 89754 > matrix_064.out

In [8]:
for d in sorted(dirs):
    #print d
    num = d[-3:]
    print num
    text ='''#!/bin/bash
#$ -N %s 
#$ -V
#$ -j y
#$ -cwd

cd %s
./bayenv2 -i UnbinnedImputedSNPSFILE%s_noHEADERIDX.txt -p 8 -k 100000 -r %s > matrix_%s.out

''' % ('mat%s' % str(num),
       d,
       str(num),
       str(random.sample(range(100000),1)[0]),
       str(num)
      )
    #print text
    filE = op.join(op.dirname(d),'M-est/get_%s_matrix.sh' % str(num))
    with open(filE,'w') as o:
        o.write("%s" % text)
    #break

000
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099


In [131]:
#qsub the .sh files

In [132]:
len(dirs)

100

In [5]:
#check on progress
done_mats = []
statslst = []
for d in dirs:
    files = ls(d)
    f = op.join(d,[f for f in files if 'matrix' in f][0])
    stats = os.stat(f)
    if stats.st_size > 0:
        #print f
        #print stats.st_size
        done_mats.append(f)
        statslst.append(stats.st_size)
len(done_mats)

100

In [6]:
#take a look at the contents of one of the finished files, making sure it reached the 100,000th chain ...
    #... by reading lines and making sure the output is an 8x8 matrix (npops = 8)
matrixfile = open(done_mats[14])
r = matrixfile.readlines()
for line in r[2005:][:-1]:
    print line

1.090341e-01	1.042588e-01	1.088373e-01	1.060631e-01	1.054348e-01	1.083376e-01	1.105650e-01	1.094948e-01	

1.042588e-01	9.986463e-02	1.041556e-01	1.015014e-01	1.008998e-01	1.036784e-01	1.058093e-01	1.047856e-01	

1.088373e-01	1.041556e-01	1.088193e-01	1.059579e-01	1.053299e-01	1.082308e-01	1.104555e-01	1.093869e-01	

1.060631e-01	1.015014e-01	1.059579e-01	1.033475e-01	1.026461e-01	1.054722e-01	1.076402e-01	1.065983e-01	

1.054348e-01	1.008998e-01	1.053299e-01	1.026461e-01	1.021259e-01	1.048468e-01	1.070020e-01	1.059667e-01	

1.083376e-01	1.036784e-01	1.082308e-01	1.054722e-01	1.048468e-01	1.078231e-01	1.099488e-01	1.088842e-01	

1.105650e-01	1.058093e-01	1.104555e-01	1.076402e-01	1.070020e-01	1.099488e-01	1.122998e-01	1.111232e-01	

1.094948e-01	1.047856e-01	1.093869e-01	1.065983e-01	1.059667e-01	1.088842e-01	1.111232e-01	1.101376e-01	



### Step 5 - split the SNPSFILEs

In [9]:
sorted(dirs)

['/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null001',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null002',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null003',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null004',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null005',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null006',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null007',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null008',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null009',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null010',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null011',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null012',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null013',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null014',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null015',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null016',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null017',
 '/home/li

In [53]:
#get a list of the loci we want to run - created in 08_bayenv2
nullf = '/home/lindb/wbp/bayenv2/results/uniq_loci_bfgreaterthan3and10worst.txt'
nu
lldf = pd.read_csv(nullf,header=0,sep='\t')
nulldf.head()

,topandlowestBFs
0,NODE_1386958_length_95_cov_1.010526_23
1,NODE_1157581_length_98_cov_2.000000_65
2,NODE_1918081_length_96_cov_1.020833_21
3,NODE_1699474_length_93_cov_1.021505_62
4,NODE_572919_length_82_cov_1.597561_54


In [54]:
len(nulldf)

2003

In [16]:
for d in dirs:
    snpsdir = op.join(d,'snps')
    os.makedirs(snpsdir)

In [ ]:
#godzilla

In [57]:
rc = Client(profile="xmn")
dview = rc[:]
lview = rc.load_balanced_view()
#len(dview)
len(lview)

5

In [60]:
def write_SNPFILEs(two):
    d,nulldf = two
    import os
    import os.path as op
    import sys
    import socket
    import stopwatch
    from subprocess import Popen, PIPE
    import tempfile
    import shutil
    import pandas as pd
    import numpy as np
    
    print d
    snpsdir = op.join(d,'snps')
    
    files = ls(d)
    if len(files) == 0:
        return 're-queue'
    f = [f for f in files if 'HEADERROW' in f][0]
    print f
    df = pd.read_csv(op.join(d,f),header=0,index_col=0,sep='\t')
    data = df[df.index.isin(nulldf['topandlowestBFs'].tolist())]
    
    loccount = 0
    for locus in np.unique(data.index).tolist():
        filE = op.join(snpsdir,locus)
        snpdf = pd.DataFrame(data.loc[locus,:])
        snpdf.to_csv(filE,header=False,index=False,sep='\t')
        loccount += 1 
        if loccount % 1000 == 0:
            print loccount 
    return loccount
dview['write_SNPSFILEs'] = write_SNPFILEs

In [50]:
dirs[0]

'/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000'

In [362]:
snp_jobs = []
for d in sorted(dirs):
    snp_jobs.append(lview.apply_async(write_SNPFILEs,[d,nulldf]))

In [372]:
count = 0
for s in snp_jobs:
    if s.ready():
        count += 1
count

100

In [377]:
for s in snp_jobs:
    if s.ready():
        print s.r

2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003


RemoteError: IndexError(list index out of range)

In [376]:
for d in sorted(dirs):
    num = op.basename(d)
    snpdir = op.join(d,'snps')

    files = ls(snpdir)
    
    print len(files),d

2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null001
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null002
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null003
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null004
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null005
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null006
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null007
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null008
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null009
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null010
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null011
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null012
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null013
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null014
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null015
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null016
2003 /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/

### Step 6  - make usable MATRIXFILEs

In [393]:
for d in sorted(dirs):
    DIR = d
    files = ls(DIR)
    if len(files)>1: #all have /snps, so len>=1
        for f in files:
            if (f.startswith('matrix')) and (f.endswith('out')):
                #print os.path.join(DIR,f)
                F = open(os.path.join(DIR,f),'r')
                g = F.readlines()
                new = f[:-4] + '.txt'
                filE = op.join(DIR,'usable_%s' % new)
                #print filE
                with open(filE,'w') as o:
                    for line in g[2005:][:-1]:
                        o.write(line)
                o.close()
                break
    else:
        print "skipped" , d 

skipped /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null064


### Step 7 - make qsub files to run bayenv2

In [9]:
snplst = []
for d in sorted(dirs):
    snpdir = op.join(d,'snps')
    files = ls(snpdir)
    if len(files) == 2003:
        [snplst.append(op.join(snpdir,f)) for f in files]
len(snplst) #one folder needs to be redone 

198297

In [10]:
bayenv_jobs = 190
num_chunks = int(np.round(200300/bayenv_jobs))
num_chunks

1054

In [11]:
chunks = [snplst[x:x+num_chunks] for x in range(0, len(snplst), num_chunks)]
chunks[0]

['/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_754072_length_98_cov_3.000000_80',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_763273_length_98_cov_1.020408_8',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_763293_length_98_cov_1.122449_103',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_763360_length_97_cov_3.381443_60',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_777994_length_98_cov_2.724490_29',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_781594_length_98_cov_1.724490_31',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_787766_length_49_cov_1.061224_34',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_788788_length_78_cov_1.064103_46',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_802648_length_90_cov_2.000000_18',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_810529_length_94_cov_2.712766_9',
 '/home/lindb/wbp/bay

In [12]:
par_dir = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/'

In [101]:
chunks[0][0]

'/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000/snps/NODE_754072_length_98_cov_3.000000_80'

In [111]:
nulldir

'/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null064'

In [36]:
bay_exe = '/home/lindb/wbp/bayenv2/bayenv2'
bay_ENV = '/home/lindb/wbp/bayenv2/ENVIRONFILE.txt'
total = 0
cpu = 0
max_cpu = 30
for i, chunk in enumerate(chunks):
    opener = op.join(par_dir, "bayenv_parallel_%s" % str(i).zfill(3))
    with open(opener, "w") as o:
        text = 'cd /home/lindb/wbp/bayenv2/\n'
        o.write("%s" % text)
        for filE in chunk:
            nulldir = op.dirname(op.dirname(filE))
            num = nulldir[-3:]
            cmd = '%s \
-i %s \
-m %s \
-e %s \
-p 8 -k 100000 -n 18 -t -X -c -f \
-r %s' % (bay_exe,
          filE,
          op.join(nulldir,'usable_matrix_%s.txt'%str(num)),
          bay_ENV,
          str(random.sample(xrange(100000),1)[0])
         )
            bayenv_cmd = cmd.split()
            if cpu == max_cpu:
                cpu = 0
            
            bayenv_cmd.insert(0, "taskset -c %d" % cpu)
            bayenv_cmd.append("-o %s" % op.join(nulldir,'bf_environfile.ENVIRONFILE.txt'))
            o.write("%s\n" % " ".join([str(x) for x in bayenv_cmd]))
            total += 1
            cpu += 1

In [ ]:
find -type f -name 'NODE*' | parallel --progress '/home/lindb/wbp/bayenv2/bayenv2 -i {} \
-m /home/lindb/wbp/bayenv2/usable_matrix.txt \
-e /home/lindb/wbp/bayenv2/ENVIRONFILE.txt \
-p 8 -k 100000 -n 18 -t -X -c -f -r 34813'

In [35]:
str(random.sample(xrange(100000),1)[0])

'35735'

In [442]:
rc = Client(profile="xmn")
dview = rc[:]
lview = rc.load_balanced_view()
#len(dview)
len(lview)

5

In [443]:
def get_line_count(f):
    res = !wc -l {f}
    return int(res[2].split()[0])
dview['get_line_count'] = get_line_count

In [444]:
par_dir

'/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/'

In [445]:
bay_par = os.listdir(par_dir)
bay_par = [os.path.join(par_dir,f) for f in bay_par if 'parallel' in f]
len(bay_par)

189

In [447]:
total = 0
for f in sorted(bay_par):
    print(f)
    total += get_line_count(f)
total #189 + 99*2003

/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_000
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_001
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_002
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_003
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_004
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_005
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_006
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_007
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_008
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_009
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_010
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_011
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_012
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_paral

198486

In [451]:
sorted(bay_par)

['/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_000',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_001',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_002',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_003',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_004',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_005',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_006',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_007',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_008',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_009',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_010',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_011',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/bayenv_parallel_012',
 '/home/lind

In [452]:
def write_qsub_files(bayenv_parallel):
    files = []
    for i, f in enumerate(bayenv_parallel):
        d = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/'
        if not os.path.exists(d):
            os.makedirs(d)
        qsub_file = os.path.join(d, "qsub_%s.sh" % str(i).zfill(3))
        files.append(qsub_file)
        with open(qsub_file, "w") as o:
            os.chmod(o.name, 0o744)
            print(o.name)
            o.write("%s\n" % "\n".join(["#!/bin/bash", 
                                        "#$ -N bayenv%d" % i,
                                        "#$ -V",
                                        "#$ -cwd",
                                        "#$ -pe smp 30",
                                        "#$ -j y",
                                        "#$ -q all.q",
                                        "unset module",
                                        "echo \"Running on $HOSTNAME\"",
                                        "cat %s | ~/bin/parallel -j 30 --progress --" % f]))
    return files
qsub_files = write_qsub_files(bay_par)

/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_000.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_001.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_002.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_003.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_004.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_005.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_006.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_007.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_008.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_009.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_010.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_011.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_012.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_013.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_014.sh
/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/qsubs/qsub_015.sh
/home/lindb/wbp/bayenv2/

    !ls /home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/parallel/ | parallel 'qsub {}'

In [8]:
DIR = '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/'
dirs = ls(DIR)
dirs = [op.join(DIR,d) for d in dirs if 'null' in d]
len(dirs)

100

In [112]:
count = 0
for d in dirs:
    files = ls(d)
    if 'bf_environfile.ENVIRONFILE.txt.bf' in files:
        #print d
        count += 1
count

100

# Explore distributions

In [113]:
nulldf.head()

,topandlowestBFs
0,NODE_1386958_length_95_cov_1.010526_23
1,NODE_1157581_length_98_cov_2.000000_65
2,NODE_1918081_length_96_cov_1.020833_21
3,NODE_1699474_length_93_cov_1.021505_62
4,NODE_572919_length_82_cov_1.597561_54


In [114]:
dirs

['/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null000',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null001',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null002',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null003',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null004',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null005',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null006',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null007',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null008',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null009',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null010',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null011',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null012',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null013',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null014',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null015',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null016',
 '/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null017',
 '/home/li

In [116]:
#get the environfile
envdf = pd.read_csv('/home/lindb/wbp/bayenv2/ENVIRONFILE_headerIDX.txt',header=0,index_col=0,sep='\t')
envdf.head()

,Dicks_Pass,Freel_Peak,Heavenly,Little_Round_Top,Mt_Rose_Ophir,Rifle_Peak,Snow_Valley_Peak,West_Shore_Peaks,Unnamed: 9
AWS0-25,-0.203570,-0.389380,-1.384005,0.474087,0.430367,0.299207,1.982420,-1.209126,NaN
AWS0-50,-0.035349,-0.578405,-1.156498,0.239933,-0.030344,0.510211,2.176919,-1.126467,NaN
Ann-ppt,1.621491,-0.620322,-1.416888,0.058608,-0.059028,0.260270,-1.124478,1.280346,NaN
CEC,-0.519958,-0.162005,-0.519958,2.565848,0.195949,-0.519958,-0.519958,-0.519958,NaN
Clay,-0.111761,-0.739189,-0.049018,2.429323,-1.209760,-0.033332,-0.017646,-0.268618,NaN


In [118]:
#assign column value for each environmental variable, to be used to name columns in bf_ENVIRONFILE.txt
envDict = OrderedDict()
for i, env in enumerate(envdf.index):
    #print i,env
    envDict[env] = [(i*3),(i*3)+1,(i*3)+2]
envDict

OrderedDict([('AWS0-25', [0, 1, 2]),
             ('AWS0-50', [3, 4, 5]),
             ('Ann-ppt', [6, 7, 8]),
             ('CEC', [9, 10, 11]),
             ('Clay', [12, 13, 14]),
             ('Elev', [15, 16, 17]),
             ('GDD-Aug', [18, 19, 20]),
             ('GDD-May', [21, 22, 23]),
             ('Lat', [24, 25, 26]),
             ('Lon', [27, 28, 29]),
             ('Max-rad-input', [30, 31, 32]),
             ('Rock-cov', [33, 34, 35]),
             ('Sand', [36, 37, 38]),
             ('Silt', [39, 40, 41]),
             ('Tmax-July', [42, 43, 44]),
             ('Tmin-Jan', [45, 46, 47]),
             ('WC15Bar', [48, 49, 50]),
             ('WC3rdbar', [51, 52, 53])])

In [122]:
orderDict = {}
orderDict[0] = 'BF'
orderDict[1] = 'rho'
orderDict[2] = 'pearson'

In [124]:
#get a dictionary to name columns in bf_environfile.ENVIRONFILE
colDict = OrderedDict()
for env,cols in envDict.items():
    #print env,cols
    colCount = 0
    for col in cols:
        sub = '_'.join([str(env), orderDict[colCount]]) 
        colDict[col] = sub
        colCount += 1
colDict

OrderedDict([(0, 'AWS0-25_BF'),
             (1, 'AWS0-25_rho'),
             (2, 'AWS0-25_pearson'),
             (3, 'AWS0-50_BF'),
             (4, 'AWS0-50_rho'),
             (5, 'AWS0-50_pearson'),
             (6, 'Ann-ppt_BF'),
             (7, 'Ann-ppt_rho'),
             (8, 'Ann-ppt_pearson'),
             (9, 'CEC_BF'),
             (10, 'CEC_rho'),
             (11, 'CEC_pearson'),
             (12, 'Clay_BF'),
             (13, 'Clay_rho'),
             (14, 'Clay_pearson'),
             (15, 'Elev_BF'),
             (16, 'Elev_rho'),
             (17, 'Elev_pearson'),
             (18, 'GDD-Aug_BF'),
             (19, 'GDD-Aug_rho'),
             (20, 'GDD-Aug_pearson'),
             (21, 'GDD-May_BF'),
             (22, 'GDD-May_rho'),
             (23, 'GDD-May_pearson'),
             (24, 'Lat_BF'),
             (25, 'Lat_rho'),
             (26, 'Lat_pearson'),
             (27, 'Lon_BF'),
             (28, 'Lon_rho'),
             (29, 'Lon_pearson'),
             (

In [235]:
op.basename(d)[-3:]

'099'

In [248]:
d

'/home/lindb/wbp/bayenv2/nullruns/SNPSFILEs/null099'

In [251]:
dirs = sorted(dirs)

In [252]:
#look to see how many loci finished bayenv2 runs
for d in dirs:
    num = d.split("/")[-1][-3:]
    snpdir = os.path.join(d,'snps')
    bffile = os.path.join(d,'bf_environfile.ENVIRONFILE.txt.bf')
    bfdf = pd.read_csv(bffile,header=None,sep="\t")        
    print num,len(bfdf.index)

000 1817
001 1601
002 1751
003 1789
004 1793
005 1709
006 1811
007 1643
008 1982
009 1604
010 2003
011 1833
012 1602
013 1606
014 1919
015 1789
016 1793
017 1793
018 1979
019 1813
020 1601
021 1815
022 1962
023 1663
024 1918
025 2003
026 1727
027 1813
028 1625
029 1997
030 1798
031 1621
032 1961
033 1871
034 1816
035 1602
036 1811
037 1811
038 1841
039 2003
040 1793
041 1619
042 1812
043 1818
044 1811
045 1601
046 1883
047 1961
048 1835
049 1787
050 1799
051 1791
052 1643
053 1943
054 1891
055 1603
056 1815
057 1811
058 1841
059 1996
060 1607
061 1811
062 1813
063 1601
064 1607
065 1816
066 1692
067 1721
068 1649
069 1766
070 1820
071 1807
072 1811
073 1967
074 1661
075 1601
076 1811
077 1673
078 2003
079 1765
080 1601
081 1799
082 1617
083 1817
084 1943
085 1793
086 1699
087 1605
088 1649
089 1763
090 1601
091 1607
092 1601
093 1601
094 1811
095 1929
096 2003
097 1793
098 1793
099 1764


# fix missing loci

In [273]:
#get a list of snps that should have been done
snps = pd.read_csv('/home/lindb/wbp/bayenv2/results/uniq_loci_bfgreaterthan3and10worst.txt')['topandlowestBFs'].tolist()
len(snps)

2003

In [282]:
#get a list of snps left to do
redos = OrderedDict()
for d in sorted(dirs):
    num = d.split("/")[-1][-3:]
    snpdir = os.path.join(d,'snps')
    
    bffile = os.path.join(d,'bf_environfile.ENVIRONFILE.txt.bf')
    bfdf = pd.read_csv(bffile,header=None,sep="\t")
    loci = [bfdf.loc[row,0].split("/")[-1] for row in bfdf.index]
    
    redosnps = set(snps)-set(loci)
    redos
    print num,len(redosnps)
    redos[num] = list(redosnps)
    
    if len(loci) > len(snps):
        print 'uh oh'
print 'len(redo.keys()) =',len(redos.keys())

000 186
001 402
002 252
003 214
004 210
005 294
006 192
007 360
008 21
009 399
010 0
011 170
012 401
013 397
014 84
015 214
016 210
017 210
018 24
019 190
020 402
021 188
022 41
023 340
024 85
025 0
026 276
027 190
028 378
029 6
030 205
031 382
032 42
033 132
034 187
035 401
036 192
037 192
038 162
039 0
040 210
041 384
042 191
043 185
044 192
045 402
046 120
047 42
048 168
049 216
050 204
051 212
052 360
053 60
054 112
055 400
056 188
057 192
058 162
059 7
060 396
061 192
062 190
063 402
064 396
065 187
066 311
067 282
068 354
069 237
070 183
071 196
072 192
073 36
074 342
075 402
076 192
077 330
078 0
079 238
080 402
081 204
082 386
083 186
084 60
085 210
086 304
087 398
088 354
089 240
090 402
091 396
092 402
093 402
094 192
095 74
096 0
097 210
098 210
099 239
100


['NODE_1446206_length_102_cov_1.852941_110',
 'NODE_1450249_length_93_cov_1.000000_35',
 'NODE_1763496_length_92_cov_1.010870_99',
 'NODE_597115_length_95_cov_2.210526_99',
 'NODE_1386197_length_92_cov_1.956522_9',
 'NODE_885522_length_98_cov_2.132653_112',
 'NODE_1157856_length_98_cov_2.000000_37',
 'NODE_722582_length_56_cov_2.375000_48',
 'NODE_172703_length_96_cov_1.135417_21',
 'NODE_965429_length_101_cov_1.772277_27',
 'NODE_1105096_length_90_cov_1.000000_31',
 'NODE_1977307_length_98_cov_2.000000_16',
 'NODE_1238481_length_53_cov_1.226415_55',
 'NODE_1548743_length_76_cov_1.013158_45',
 'NODE_1054180_length_94_cov_1.000000_52',
 'NODE_1670849_length_102_cov_1.098039_99',
 'NODE_2112040_length_97_cov_1.010309_37',
 'NODE_1682863_length_165_cov_1.036364_19',
 'NODE_105102_length_94_cov_1.127660_94',
 'NODE_1104336_length_125_cov_1.120000_128',
 'NODE_111138_length_56_cov_1.339286_77',
 'NODE_1225576_length_98_cov_1.908163_80',
 'NODE_399735_length_98_cov_4.000000_17',
 'NODE_34441

# end of fixing missing loci

In [243]:
#get all of the bf_ENVIRONFILEs as a dataframe within a dictionary
bfDict = OrderedDict()
for d in dirs:
    if not '064' in d:
        num = d.split("/")[-1][-3:]
        snpdir = os.path.join(d,'snps')
        bffile = os.path.join(d,'bf_environfile.ENVIRONFILE.txt.bf')
        bfdf = pd.read_csv(bffile,header=None,sep="\t")        
        print num,len(bfdf.index)

000 1817
001 1601
002 1751
003 1789
004 1793
005 1709
006 1811
007 1643
008 1982
009 1604
010 2003
011 1833
012 1602
013 1606
014 1919
015 1789
016 1793
017 1793
018 1979
019 1813
020 1601
021 1815
022 1962
023 1663
024 1918
025 2003
026 1727
027 1813
028 1625
029 1997
030 1798
031 1621
032 1961
033 1871
034 1816
035 1602
036 1811
037 1811
038 1841
039 2003
040 1793
041 1619
042 1812
043 1818
044 1811
045 1601
046 1883
047 1961
048 1835
049 1787
050 1799
051 1791
052 1643
053 1943
054 1891
055 1603
056 1815
057 1811
058 1841
059 1996
060 1607
061 1811
062 1813
063 1601
065 1816
066 1692
067 1721
068 1649
069 1766
070 1820
071 1807
072 1811
073 1967
074 1661
075 1601
076 1811
077 1673
078 2003
079 1765
080 1601
081 1799
082 1617
083 1817
084 1943
085 1793
086 1699
087 1605
088 1649
089 1763
090 1601
091 1607
092 1601
093 1601
094 1811
095 1929
096 2003
097 1793
098 1793
099 1764


In [242]:
d.split("/")[-1][-3:]

'099'

In [232]:
len(bfdf.index)

1764

In [153]:
#get all of the bf_ENVIRONFILEs as a dataframe within a dictionary
bfDict = OrderedDict()
for d in sorted(dirs):
    if not '064' in d:
        num = d.split("/")[-1][-3:]
        print num
        snpdir = os.path.join(d,'snps')
        bffile = os.path.join(d,'bf_environfile.ENVIRONFILE.txt.bf')
        bfdf = pd.read_csv(bffile,header=None,sep="\t")
        
        loci = []
        for row in bfdf.index:
            locus = bfdf.loc[row,0].split("/")[-1]
            loci.append(locus)
        bfdf = pd.DataFrame(bfdf.loc[:,[x for x in bfdf.columns[1:-1]]])
        bfdf.columns = [x for x in range(len(bfdf.columns))]
        bfdf.columns = [colDict[col]+'_%s'%num for col in bfdf.columns]
        bfdf.index = [locus for locus in loci]
        bfDict[num] = bfdf

000
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099


In [263]:
bfDict.keys()

[]

In [154]:
len(bfDict.keys())

99

In [220]:
bfDict[90]['WCrdbar_BF_090'].head()

,AWS0-25_BF_090,AWS0-25_rho_090,AWS0-25_pearson_090,AWS0-50_BF_090,AWS0-50_rho_090,AWS0-50_pearson_090,Ann-ppt_BF_090,Ann-ppt_rho_090,Ann-ppt_pearson_090,CEC_BF_090,...,Tmax-July_pearson_090,Tmin-Jan_BF_090,Tmin-Jan_rho_090,Tmin-Jan_pearson_090,WC15Bar_BF_090,WC15Bar_rho_090,WC15Bar_pearson_090,WC3rdbar_BF_090,WC3rdbar_rho_090,WC3rdbar_pearson_090
NODE_754072_length_98_cov_3.000000_80,0.058549,-0.033333,-0.034345,0.058844,-0.031328,-0.037116,0.046275,0.073183,0.094893,0.062336,...,-0.119680,0.044747,0.014662,0.037509,0.060232,0.006266,-0.036464,0.059608,-0.000125,-0.041672
NODE_763273_length_98_cov_1.020408_8,0.055776,-0.003384,-0.007851,0.046055,-0.018922,-0.008699,0.050580,-0.031328,-0.017933,0.045826,...,0.011599,0.099947,-0.022682,0.006487,0.039874,0.001128,-0.010722,0.039222,0.005890,-0.001490
NODE_763293_length_98_cov_1.122449_103,0.055648,-0.011028,-0.021523,0.066474,-0.009023,-0.015236,0.050639,-0.026190,-0.006315,0.040539,...,-0.104000,0.040860,-0.116170,-0.040041,0.067672,0.055514,-0.014424,0.063678,0.058772,-0.006445
NODE_763360_length_97_cov_3.381443_60,0.036168,-0.045238,-0.036396,0.039959,-0.063033,-0.055912,0.039007,-0.026692,0.018370,0.034087,...,-0.120510,0.034913,-0.063409,-0.049394,0.040652,0.006391,-0.054211,0.038311,0.006015,-0.043654
NODE_777994_length_98_cov_2.724490_29,0.070259,0.015288,-0.001176,0.070956,0.045489,0.035767,0.070551,0.155010,0.140520,0.033130,...,-0.206880,0.075289,-0.058145,0.087857,0.060095,0.088847,0.032495,0.051274,0.072180,0.003877


In [230]:
len(bfDict[30].index)

1798

In [166]:
#make dataframes for each env that contain all BFs across runs
envbfDict = OrderedDict()
for env in envdf.index: # for each environmental variable
#    print env
    dfcount = 0
    for run in bfDict.keys(): #for each bf_ENVIRONFILE
        col = '%s_BF_%s' % (env,str(run).zfill(3))
        df = pd.DataFrame(bfDict[run][col])
        if dfcount == 0:
            newdf = df
        else:
            newdf = pd.merge(newdf,df,left_index=True,right_index=True)
        dfcount += 1
        #break
    filE = '/home/lindb/wbp/bayenv2/nullruns/results/%s_all_BFs.txt' % env
    newdf.to_csv(filE,header=True,index=True,sep="\t")
    envbfDict[env] = newdf

In [226]:
pd.DataFrame(bfDict[run][col]).sort_values(by='WC3rdbar_BF_099',ascending=1)

,WC3rdbar_BF_099
NODE_386699_length_148_cov_1.243243_157,0.028237
NODE_151928_length_106_cov_1.179245_106,0.028884
NODE_2113184_length_64_cov_1.000000_28,0.028965
NODE_1681521_length_95_cov_2.800000_32,0.029200
NODE_2056691_length_47_cov_1.000000_45,0.029232
NODE_840142_length_92_cov_1.010870_57,0.029425
NODE_662539_length_86_cov_9.058140_45,0.029563
NODE_1174811_length_98_cov_2.000000_15,0.029564
NODE_97950_length_140_cov_4.485714_72,0.029618
NODE_152272_length_96_cov_1.156250_29,0.029710


In [167]:
#get BFs from the empirical (true data) run of bayenv2
empBFs = pd.read_csv('/home/lindb/wbp/bayenv2/results/BF_all.txt',header=0,index_col=0,sep="\t")
empBFs.head()

,AWS0-25_BF,AWS0-50_BF,Ann-ppt_BF,CEC_BF,Clay_BF,Elev_BF,GDD-Aug_BF,GDD-May_BF,Lat_BF,Lon_BF,Max-rad-input_BF,Rock-cov_BF,Sand_BF,Silt_BF,Tmax-July_BF,Tmin-Jan_BF,WC15Bar_BF,WC3rdbar_BF
NODE_1022341_length_110_cov_1.572727_121,0.039412,0.044012,0.043089,0.069205,0.065406,0.040421,0.026872,0.035479,0.048453,0.040624,0.059265,0.060311,0.050194,0.052312,0.043438,0.030228,0.052632,0.043833
NODE_1022323_length_92_cov_1.141304_34,0.038105,0.035717,0.043443,0.043571,0.032902,0.056315,0.053381,0.036369,0.041288,0.046238,0.036812,0.047657,0.031352,0.029938,0.046391,0.564850,0.039821,0.039301
NODE_1000053_length_94_cov_1.744681_25,0.069298,0.045437,0.040234,0.043974,0.031790,0.035175,0.039015,0.030961,0.036205,0.107700,0.060989,0.050068,0.040488,0.047824,0.301050,0.031764,0.044626,0.050776
NODE_1037913_length_76_cov_1.210526_45,0.151090,0.088241,0.046748,0.124360,0.041694,0.060639,0.074511,0.119890,0.042254,0.041710,0.044512,0.071720,0.067409,0.058143,0.044706,0.049883,0.050024,0.054170
NODE_1037972_length_59_cov_1.237288_42,0.072104,0.077983,0.047167,0.055690,0.067452,0.061527,0.051156,0.050269,0.071740,0.044611,0.086973,0.040416,0.083884,0.073501,0.038904,0.063020,0.056250,0.050188


In [171]:
len(envbfDict['Elev'].columns) #100 null runs

99

In [206]:
#get the empirically IDed snps from bayenv2 (BF >= 3)
snpdir = '/home/lindb/wbp/bayenv2/results/'
snpfiles = [op.join(snpdir,f) for f in ls(snpdir) if '3and10' in f and 'uniq' not in f]

snpDict = OrderedDict()
for f in snpfiles:
    env = op.basename(f).split("_")[0]
    snpDict[env] = OrderedDict()
    df = pd.read_csv(f,header=0,index_col=0,sep="\t")
    
    snpDict[env]['best'] = df[df["%s_BF"%env] >=3].index
    snpDict[env]['worst']= df[df["%s_BF"%env] < 3].index
    
    print env,len(snpDict[env]['best'])+len(snpDict[env]['worst'])

AWS0-25 146
AWS0-50 144
Ann-ppt 121
CEC 155
Clay 143
Elev 142
GDD-Aug 162
GDD-May 140
Lat 108
Lon 133
Max-rad-input 130
Rock-cov 130
Sand 117
Silt 127
Tmax-July 138
Tmin-Jan 140
WC15Bar 143
WC3rdbar 169


In [ ]:
#get dataframes with just the snps IDed from empirical bayenv2 runs
newDict = OrderedDict()
for env in envbfDict.keys():
    df = envbfDict[env]
    df.index = [str(x[2:]) for x in envbfDict[env].index]
    newDict[env] = df[df.index.isin(snpDict[env])]   